## protocol_pdf2table 
converts PDF protocols from ISU skating competitions and converts them to readable DataFrames or CSV files. protocol_pdf2table uses tabula-py to read tables from PDFs.

### Goal
My goal for this project is to make skating data accessible. I want to have as much data as possible about all things related to figure skating, and I want this to be easily accessible and usable for everyone.  

Then, we can do cool things with this data. Some example ideas are determing racial/gender/nation/other inequity in judging; determining the effect PC versus TE scores have on outcomes; quantifying the "quad effect"; and predicting future champions (e.g. who will make the next Olympics).

### Setup

In [1]:
# Upgrade pip, then 
#Install pandas, tabula-py, and pdfplumber packages

import sys
!{sys.executable} -m pip install --upgrade pip --user
!{sys.executable} -m pip install pandas --user
!{sys.executable} -m pip install tabula-py --user
!{sys.executable} -m pip install pdfplumber --user

     |████████████████████████████████| 42 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 2.1 MB/s eta 0:00:01
Using legacy 'setup.py install' for pdfplumber, since package 'wheel' is not installed.
    Running setup.py install for pdfplumber ... done


In [2]:
# import tabula, re, pandas as pd, numpy as np, and pdfplumber

import tabula            #tabula.environment_info()
import re
import pandas as pd
import numpy as np
import pdfplumber

### Current Work: 
Import all data from a single event: Pairs SP Worlds 2019 into clean data frames

#### Set the event url as the pdf_path

In [3]:
pdf_path = "http://www.isuresults.com/results/season1819/wc2019/wc2019_protocol.pdf"
pdf_filepath = "/Users/spencersimon/Documents/Coding/skating_projects/Protocol_PDFs/wc2019_protocol.pdf"

# Might need to add to or change this; not sure yet
officials_pdf_path = "https://www.isu.org/figure-skating/rules/fsk-communications/21860-isu-communication-2273/file"

#### Set constants & variables for reading tables from the PDF

In [4]:
# Set common heights and width (in PDF pixels)
width = 499             # Width of the whole protocol table
height_header = 42      # Height of the header table of the protocol
height_ded = 11         # Height of the deductions table of the protocol
space_h2b = 4           # Space between the header and body tables
space_b2d = 3           # Space between the body and deduction tables
space_protocols = 14    # Space between protocols (bottom of deductions to top of next header)


# Set height for body of protocol for each event
# approximate formula: height = 94 + 9*NumberOfElements

height_ws = 157         # Height of protocol body for Womens Short Program (7 elements)
height_ms = 157         # Height of protocol body for Mens Short Program (7 elements)
height_ps = 157         # Height of protocol body for Pairs Short Program (7 elements)
height_ds = 139         # Height of protocol body for Dance Short Program (5 elements)
height_wf = 200         # Height of protocol body for Womens Free Program (12 elements)
height_mf = 200         # Height of protocol body for Mens Free Program (12 elements)
height_pf = 192         # Height of protocol body for Pairs Free Program (11 elements)
height_df = 183         # Height of protocol body for Dance Free Program (10 elements)


# Set coordinates
left = 48               # Left x-coordinate of protocol tables
right = 547             # Right x-coordinate of protocol tables

left_official = 82      # Left x-coordinate of official tables
right_official = 494    # Right x-coordinate of official tables
top_judge = 154
bottom_judge = 364
top_tech = 383
bottom_tech = 466

yH0 = 94                # Top y-coordinate of the 1st header on the page

#### Function definitions 
based on above constants and variables, to read in tables from PDF

##### coords function

In [5]:
# Function: Find the coords of a protocol table or official table on the pdf, 
# for Worlds 2019
# ADAPT to work for judges names, and other tables included too
def coords(event, part, number=0):
    """ # Take in event STR (e.g. 'Pairs Short'), protocol part STR (e.g. 'header'), 
        # and number on page INT (0, 1, or 2). Default number = 0.
        # Return coordinates of that table in form [top, left, bottom, right] """
    
    # Check if Officials first. If so, return area for that part of officials table
    if part == "official":
        if number == 0:               # Area for Panel of Judges
            area = [154,82,364,494]
            return area
        elif number == 1:             # Area for Technical Panel
            area = [383,82,466,494]
            return area
        else:
            raise Exception("Number not recognized for Officials table. Use '0' for " + 
                        "Panel of Judges, or '1' for Technical panel")
    # Check the event, and return the corresponding height of the body of that protocol
    # *Change to use regex to be more forgiving to input*        
    elif event == "Pairs Short":     # if re.search("pattern", event)
        height_b = height_ps
    elif event == "Pairs Free":
        height_b = height_pf
    elif event == "Dance Short":
        height_b = height_ds
    elif event == "Dance Free":
        height_b = height_df
    elif event == "Womens Short":
        height_b = height_ws
    elif event == "Womens Free":
        height_b = height_wf
    elif event == "Mens Short":
        height_b = height_ms
    elif event == "Mens Free":
        height_b = height_mf
    else:
        raise Exception("Event name not recognized. Use Womens Short, " + 
                        "Mens Free, or Dance Short, etc.")
    
    # Check for the protocol part, and return top and bottom coords for 1st of that part 
    # on the page **Change to use regex to be more forgiving to input**?
    if part == "header":
        top = yH0
        bottom = yH0+height_header
    elif part == "body":
        top = yH0+height_header+space_h2b
        bottom = top+height_b
    elif part == "ded":
        top = yH0+height_header+space_h2b+height_b+space_b2d
        bottom = top+height_ded
    else:
        raise Exception("Protocol part not recognized. Use 'header', 'body', " + 
                        "'ded', or 'official' for protocol part.")
    
    # Check for number, and adjust top and bottom values to match
    if number == 0:
        area = [top,left,bottom,right]
    elif number == 1:
        top = top + height_header + height_b + height_ded + space_h2b + space_b2d + space_protocols
        bottom = bottom + height_header + height_b + height_ded + space_h2b + space_b2d + space_protocols
        area = [top,left,bottom,right]
    elif number == 2:
        top = top + 2*(height_header+height_b+height_ded+space_h2b+space_b2d+space_protocols)
        bottom = bottom + 2*(height_header+height_b+height_ded+space_h2b+space_b2d+space_protocols)
        area = [top,left,bottom,right]
    else:
        raise Exception("Protocol number is too high. Number must be 0, 1, or 2.")
        
    return area


# ADD THIS?
# Function:
# Find body height based on event, or based on number of elements?

##### pdf2table function

In [6]:
# Function: Convert all protocols within an event (Pairs SP for WC2019) to a 
# list of dataframes
def pdf2table(event, part, numSkaters=0, page_number=0):
    """ Takes in event STR (e.g. 'Pairs Short'), protocol part STR (e.g. 'header'), 
    number of skaters in the event INT, 
    and initial page number INT (where the 1st protocol of the even is located).
    Returns a list of data frames of the given part for all of the protocols in the event. """
    
    df_part = []                # initial (empty) list, where protocols will be added
    
    if part == "official":
        df_part.append(tabula.read_pdf(pdf_path, pages=page_number, stream=True,
                       area=coords(event, part, 0))[0])
        df_part.append(tabula.read_pdf(pdf_path, pages=page_number, stream=True,
                       area=coords(event, part, 1))[0])
        return df_part
    
    protocol_nums = range(0, numSkaters)    # Create a range to loop over
    
    for i in protocol_nums:
        print("Working...converting protocal ",i+1)
        
        # Number on page (0, 1, or 2 since there are 3 protocols per page)
        protocol_number = i % 3   
    
        df_part.append(tabula.read_pdf(pdf_path, pages=page_number, stream=True, 
                        area=coords(event, part, protocol_number))[0])
    
        # Page Number
        if i % 3 == 2:
            page_number = page_number+1  # increase page number by 1 after every 3rd protocol
    
    return df_part

##### element_details function

In [7]:
# Function: Given output from pdf2table function (df_body), return the elem_code, Name,
# Type, Detailed_Type, lvl, and info
# Right now, this just works for current scale of values (2020). Check and update to 
# work for older protocols as well.
# KEEP WORKING HERE: -Add in pairs lifts  -Add in dance elements
#                    -Add in info (under-rotated, downgraded, etc.) to Element_Name
#                    -Add in base value here? And remove elsewhere?
def element_details(df, skaterNumber, elementNumber):
    """Takes in a df_body dataframe, skater number, and element number. Returns: 
    [Element_Code, Element_Name, Element_Type, Element_Level]"""
    #####################################
    
    # Get the string for the element based on skaterNumber and elementNumber
    elemstring = df[skaterNumber-1].iloc[elementNumber][0]
    #print("Element String: ")
    #print(elemstring)
    
    # Pull out element code from the string
    match = re.search("\s(.*)", elemstring) 
    
    Element_Code = match.group(1)    # Element_Code is a string for the element, e.g. '3Tw4'
    #print("Element Code: ")
    #print(Element_Code)
    
    #####################################
    # Find the element level
    if re.search("BV?$", Element_Code):   # If element_code ends in 'B' or 'BV', lvl is Base
        Element_Level = "Base"
    elif re.search("1V?$", Element_Code):
        Element_Level = 1
    elif re.search("2V?$", Element_Code):
        Element_Level = 2
    elif re.search("3V?$", Element_Code):
        Element_Level = 3
    elif re.search("4V?$", Element_Code):
        Element_Level = 4
    else: 
        Element_Level = None
    ######################################
    ######################################
    ### Find the element type, detailed_type, and name
    
    # Need to add in Dance elements
    
    # Spins
    if re.search("Sp", Element_Code):
        Element_Type = "Spin"
        if re.search("^PSp", Element_Code):
            Element_Name = "Pair Spin"
        elif re.search("^PCoSp", Element_Code):
            Element_Name = "Pair Combination Spin"
        elif re.search("^CCoSp", Element_Code):
            Element_Name = "Change Foot Combination Spin"
        elif re.search("^FCCoSp", Element_Code):
            Element_Name = "Flying Change Foot Combination Spin"
        elif re.search("^CoSp", Element_Code):
            Element_Name = "Combination Spin"
        elif re.search("^FCoSp", Element_Code):
            Element_Name = "Flying Combination Spin"
        elif re.search("^CSSp", Element_Code):
            Element_Name = "Change Foot Sit Spin"
        elif re.search("^FCSSp", Element_Code):
            Element_Name = "Flying Change Foot Sit Spin"
        elif re.search("^CCSp", Element_Code):
            Element_Name = "Change Foot Camel Spin"
        elif re.search("^FCCSp", Element_Code):
            Element_Name = "Flying Change Foot Camel Spin"
        elif re.search("^CLSp", Element_Code):
            Element_Name = "Change Foot Layback Spin"
        elif re.search("^FCLSp", Element_Code):
            Element_Name = "Flying Change Foot Layback Spin"
        elif re.search("^CUSp", Element_Code):
            Element_Name = "Change Foot Upright Spin"
        elif re.search("^FCUSp", Element_Code):
            Element_Name = "Flying Change Foot Upright Spin"
        elif re.search("^FSSp", Element_Code):
            Element_Name = "Flying Sit Spin"
        elif re.search("^FCSp", Element_Code):
            Element_Name = "Flying Camel Spin"
        elif re.search("^FLSp", Element_Code):
            Element_Name = "Flying Layback Spin"
        elif re.search("^FUSp", Element_Code):
            Element_Name = "Flying Upright Spin"
        elif re.search("^SSp", Element_Code):
            Element_Name = "Sit Spin"
        elif re.search("^CSp", Element_Code):
            Element_Name = "Camel Spin"
        elif re.search("^LSp", Element_Code):
            Element_Name = "Layback Spin"
        elif re.search("^USp", Element_Code):
            Element_Name = "Upright Spin"
        else:
            raise Exception("\n Element_Name not recognized from Element_Code string." + 
                            "\n" + "skaterNumber: " + str(skaterNumber) + "\n" + 
                            "elementNumber: " + str(elementNumber) + "\n" +
                            "element_code: " + str(Element_Code))
    
    # Step Sequences
    elif re.search("StSq", Element_Code):
        Element_Type = "Step Sequence"
        Element_Name = "Step Sequence"
    # Choreographic Sequences
    elif re.search("ChSq", Element_Code):
        Element_Type = "Choreographic Sequence"
        Element_Name = "Choreographic Sequence"
        
    # Lifts ASK JACOB FOR HELP
    elif re.search("Li", Element_Code):
        Element_Type = "Lift"
        
        # Lift type (Element_Name)
        if re.search("^1", Element_Code):
            Element_Name = "Group 1 Lift (Hand-To-Armpit)"
        elif re.search("^2", Element_Code):
            Element_Name = "Group 2 Lift (Hand-To-Waist)"
        elif re.search("^3", Element_Code):
            Element_Name = "Group 3 Lift (Hand-To-Hip or Upper Leg)"
        elif re.search("^4", Element_Code):
            Element_Name = "Group 4: Press Lift"
        elif re.search("^5", Element_Code):
            if re.search("^5T", Element_Code):
                Element_Name = "Group 5: Lasso Lift (Toe)"
            elif re.search("^5S", Element_Code):
                Element_Name = "Group 5: Lasso Lift (Step)"
            elif re.search("^5B", Element_Code):
                Element_Name = "Group 5: Lasso Lift (Backward)"
            elif re.search("^5A", Element_Code):
                Element_Name = "Group 5: Lasso Lift (Axel)"
            elif re.search("^5R", Element_Code):
                Element_Name = "Group 5: Lasso Lift (Reverse)"
            else:
                raise Exception("\n Element_Name not recognized from Element_Code string: Lift." 
                                + "\n" + "skaterNumber: " + str(skaterNumber) + "\n" +
                                "elementNumber: " + str(elementNumber) + "\n" +
                                "element_code: " + str(Element_Code))
        else:
            raise Exception("\n Element_Name not recognized from Element_Code string: Lift." 
                            + "\n" + "skaterNumber: " + str(skaterNumber) + "\n" +
                            "elementNumber: " + str(elementNumber) + "\n" +
                            "element_code: " + str(Element_Code))
        
    # Death Spirals
    elif re.search("Ds", Element_Code):
        Element_Type = "Death Spiral"
        if re.search("BoDs", Element_Code):
            Element_Name = "Back Outside Death Spiral"
        elif re.search("BiDs", Element_Code):
            Element_Name = "Back Inside Death Spiral"
        elif re.search("FoDs", Element_Code):
            Element_Name = "Forward Outside Death Spiral"
        elif re.search("FiDs", Element_Code):
            Element_Name = "Forward Inside Death Spiral"
        else:
            raise Exception("\n Element_Name not recognized from Element_Code string: Death Spiral." 
                            + "\n" + "skaterNumber: " + str(skaterNumber) + "\n" +
                            "elementNumber: " + str(elementNumber) + "\n" +
                            "element_code: " + str(Element_Code))
    elif re.search("PiF", Element_Code):
        Element_Type = "Death Spiral"
    
    # Twists
    elif re.search("Tw", Element_Code):
        Element_Type = "Twist"
        # Get Twist Rotation:
        if re.search("^1", Element_Code):
            Element_Name = "Single Twist"
        elif re.search("^2", Element_Code):
            Element_Name = "Double Twist"
        elif re.search("^3", Element_Code):
            Element_Name = "Triple Twist"
        elif re.search("^4", Element_Code):
            Element_Name = "Quadruple Twist"
        elif re.search("^5", Element_Code):
            Element_Name = "Quintuple Twist"
        else:
            raise Exception("\n Element_Name not recognized from Element_Code string: No leading number for twist rotation." 
                            + "\n" + "skaterNumber: " + str(skaterNumber) + "\n" +
                            "elementNumber: " + str(elementNumber) + "\n" +
                            "element_code: " + str(Element_Code))
    # Jumps and Throws
    elif re.search("(A|T|S|F|Lo|Lz|Eu)", Element_Code): # Jump is last bc TRUE if matches "S"
        # Test if Jump or Throw and assign element_type
        if re.search("Th", Element_Code):
            Element_Type = "Throw"
        else:
            Element_Type = "Jump"
        # Test if Combo Jump
        if re.search("\+", Element_Code):
            Element_Name = "Combination Jump"
            # *Add Triple/Double/etc. to Combo Jump Name*
        else:
            # If not combo jump: Get jump rotation
            if re.search("^1", Element_Code):
                Element_Name = "Single"
            elif re.search("^2", Element_Code):
                Element_Name = "Double"
            elif re.search("^3", Element_Code):
                Element_Name = "Triple"
            elif re.search("^4", Element_Code):
                Element_Name = "Quadruple"
            elif re.search("^5", Element_Code):
                Element_Name = "Quintuple"
            else:
                raise Exception("\n Element_Name not recognized from Element_Code string: No leading number for jump rotation." 
                                + "\n" + "skaterNumber: " + str(skaterNumber) + "\n" +
                                "elementNumber: " + str(elementNumber) + "\n" +
                                "element_code: " + str(Element_Code))
                
            # Add jump type to jump element name
            if re.search("[0-9]A", Element_Code):
                Element_Name = Element_Name + " Axel"
            elif re.search("[0-9]S", Element_Code):
                Element_Name = Element_Name + " Sal Chow"
            elif re.search("[0-9]T", Element_Code):
                Element_Name = Element_Name + " Toe Loop"
            elif re.search("[0-9]Lo", Element_Code):
                Element_Name = Element_Name + " Loop"
            elif re.search("[0-9]F", Element_Code):
                Element_Name = Element_Name + " Flip"
            elif re.search("[0-9]Lz", Element_Code):
                Element_Name = Element_Name + " Lutz"
            elif re.search("[0-9]Eu", Element_Code):
                Element_Name = Element_Name + " Euler"
            else:
                raise Exception("\n Element_Name not recognized from Element_Code string: Jump type not recognized." 
                                + "\n" + "skaterNumber: " + str(skaterNumber) + "\n" +
                                "elementNumber: " + str(elementNumber) + "\n" +
                                "element_code: " + str(Element_Code))
            
            # Assign "Throw" to Element Name if it is a throw
            if Element_Type == "Throw":
                Element_Name = "Throw " + Element_Name
    else: 
        raise Exception("Element_Code string did not match any Element Type." + "\n" +
                        "skaterNumber: " + str(skaterNumber) + "\n" + 
                        "elementNumber: " + str(elementNumber))
    ######################################
    ######################################
    # Detailed Type
    
    ######################################
    ######################################
    # Name
    
    ######################################
        
    return [Element_Code, Element_Name, Element_Type, Element_Level]

##### find_pID function

In [8]:
# Function: finds the pID based on the event_id and the skater number
def find_pID(SkaterNumber, event_id):
    """Takes in SkaterNumber INT and event_id INT. Returns the pID from dfProtocols
    of that skater in that event."""
    # Since protocols are read in order of rank, skaterNumber 1 = event_rank 1, etc.
    event_rank = SkaterNumber 
    match = dfProtocols.query('eventID == @event_id and \
                               Event_Rank == @event_rank').iloc[0][0]
    return match

#### Function Tests

In [9]:
# TEST coords
coords("Pairs Short", "header", 0)

[94, 48, 136, 547]

#### Read PDFs 
Take Protocols in and convert to Pandas DataFrames, using tabula.
tabula.read_pdf converts a pdf to a list of Pandas DataFrames

In [10]:
df_header = pdf2table("Pairs Short", "header", 19, 13) # Takes ~<1 min

Working...converting protocal  1
Working...converting protocal  2
Working...converting protocal  3
Working...converting protocal  4
Working...converting protocal  5
Working...converting protocal  6
Working...converting protocal  7
Working...converting protocal  8
Working...converting protocal  9
Working...converting protocal  10
Working...converting protocal  11
Working...converting protocal  12
Working...converting protocal  13
Working...converting protocal  14
Working...converting protocal  15
Working...converting protocal  16
Working...converting protocal  17
Working...converting protocal  18
Working...converting protocal  19


In [11]:
df_body = pdf2table("Pairs Short", "body", 19, 13) # Takes ~<1 min
# GO through everything, and update df_body, df_header, etc. in any code to adjust for
# multiple events

Working...converting protocal  1
Working...converting protocal  2
Working...converting protocal  3
Working...converting protocal  4
Working...converting protocal  5
Working...converting protocal  6
Working...converting protocal  7
Working...converting protocal  8
Working...converting protocal  9
Working...converting protocal  10
Working...converting protocal  11
Working...converting protocal  12
Working...converting protocal  13
Working...converting protocal  14
Working...converting protocal  15
Working...converting protocal  16
Working...converting protocal  17
Working...converting protocal  18
Working...converting protocal  19


In [12]:
df_ded = pdf2table("Pairs Short", "ded", 19, 13) # Takes ~<1 min

Working...converting protocal  1
Working...converting protocal  2
Working...converting protocal  3
Working...converting protocal  4
Working...converting protocal  5
Working...converting protocal  6
Working...converting protocal  7
Working...converting protocal  8
Working...converting protocal  9
Working...converting protocal  10
Working...converting protocal  11
Working...converting protocal  12
Working...converting protocal  13
Working...converting protocal  14
Working...converting protocal  15
Working...converting protocal  16
Working...converting protocal  17
Working...converting protocal  18
Working...converting protocal  19


In [13]:
dfO = pdf2table("Pairs Short", "official", page_number=11)

In [14]:
# Temporarily save df's from pdf2table call to csv's or something, and re-read them,
# so I don't have to re-run the pdf2table calls each time?

### Clean output of pdf2table tabula call

In [15]:
# TEST CELL: 
# Create list of skater names to use in dictionary


# List elems from 1 protocol
#l = [df_body[1].iloc[i+1][0] for i in range(NumElems)]

## Many of these numbers (in below dictionary definitions)
## depend on the number of elements in a program (will change correct numbers in iloc calls)
## Update to take this into account

#### Define constants

In [16]:
#Setup

NumSkaters = len(df_body)
NumOfficials = 13      # Number of officials per protocol: 9 judges + ref + TC + ATC + TS

# Finds number of elements by finding total rows in protocol body, then subtracting 
# 5 PC rows, and 2 headers and summaries (1 for elements and 1 for components)
NumElems = np.shape(df_body[14])[0] - 5 - 4
NumPC = 5

eventID = 0
competitionID = 0

#### Skaters Table
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique skIDs.

In [17]:
### dictionaries for dataframe

# Extract data from earlier tabula calls into lists to put here in dictionaries, then 
# make dataframe. This is faster than making empty dataframe and appending.


# Quantify skater international ranking, or experience level?
dSK = {'skID': [None]*NumSkaters,#[i+1 for i in range(NumSkaters)], 
       'Name': [df_header[i].iloc[3][1] for i in range(NumSkaters)], 
       'Nation': [df_header[i].iloc[3][2] for i in range(NumSkaters)],
       'Race': [None]*NumSkaters,
       'DOB': [None]*NumSkaters}



dfSkaters = pd.DataFrame(data=dSK)

# fix name and nation columns in dfSkaters
for index, row in dfSkaters.iterrows():
    if pd.isnull(row[2]):
        print(row[2])
        print("Fixing skater number " + str(index) + ": " + dfSkaters.iloc[index][1][:-4])
        dfSkaters.at[index, 'Nation'] = dfSkaters.iloc[index][1][-3:]
        dfSkaters.at[index, 'Name'] = dfSkaters.iloc[index][1][:-4] 
        
# Assign skID to dfSkaters
#dfSkaters = dfSkaters.assign(skID=(dfSkaters['Name']).astype('category').cat.codes) #Old way
dfSkaters['skID'] = dfSkaters.groupby(['Name',
                                       'Nation']).ngroup()

nan
Fixing skater number5: Aleksandra BOIKOVA / Dmitrii KOZLOVSKII
nan
Fixing skater number14: Lana PETRANOVIC / Antonio SOUZA-KORDEIRU


#### Competitions Table
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique competitionIDs.

In [18]:
# UPDATE this to read in text from pdf and put in dictionary, instead of me adding the text
dCOMPETITION = {'competitionID': [0], 
                'Competition': ["World Figure Skating Championships 2019"], 
                'Country': ["Japan"], 
                'City': ["Saitama"]}

#with pdfplumber.open(pdf_filepath) as pdf:
    #page = pdf.pages[1]
    #titleText = page.extract_text()
    ##print(page.extract_text())

dfCompetitions = pd.DataFrame(data=dCOMPETITION) # Make Dataframe from dictionary

# Assign competitionID to dfCompetitions
dfCompetitions['competitionID'] = dfCompetitions.groupby(['Competition']).ngroup()

In [19]:
dfCompetitions

,competitionID,Competition,Country,City
0,0,World Figure Skating Championships 2019,Japan,Saitama


#### Events Table
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique eventIDs.

In [20]:
# UPDATE to account for multiple events
# UPDATE Start_Time and Date to be a time format, look into this
dEVENT = {'eventID': [eventID], 
          'Event_Name': ["Pairs Short Program"], 
          'Number_of_Skaters': [NumSkaters], 
          'Start_Time': ["10:30"], # This is in the local event time zone, 24HR
          'Date': ["3/20/2019"], 
          'competitionID': [competitionID]}


dfEvents = pd.DataFrame(data=dEVENT) # Make Dataframe from dictionary
dfEvents

,eventID,Event_Name,Number_of_Skaters,Start_Time,Date,competitionID
0,0,Pairs Short Program,19,10:30,3/20/2019,0


In [21]:
dfEvents.iloc[0][0]

0

#### Protocols Table
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique pIDs.

In [22]:
dP = {'pID': [None]*NumSkaters, 
      'Event_Rank': [int(df_header[i].iloc[3][0]) for i in range(NumSkaters)], 
      'Event_StartNum': [df_header[i].iloc[3][3] for i in range(NumSkaters)], 
      'Event_Total_Score': [float(df_header[i].iloc[3][4]) for i in range(NumSkaters)], 
      'Element_Score': [float(df_header[i].iloc[3][5]) for i in range(NumSkaters)], 
      'Total_Element_Base_Value': [float(df_body[i].iloc[8][2]) for i in range(NumSkaters)], 
      'Total_GOE_Points': [round(float(df_body[i].iloc[8][15])-
                                 float(df_body[i].iloc[8][2]),3) for i in range(NumSkaters)], 
      'Component_Score': [float(df_header[i].iloc[3][6]) for i in range(NumSkaters)], 
      'Deductions': [float(df_header[i].iloc[3][7]) for i in range(NumSkaters)], 
      'J1ID': [None]*NumSkaters,  'J2ID': [None]*NumSkaters,  'J3ID': [None]*NumSkaters,
      'J4ID': [None]*NumSkaters,  'J5ID': [None]*NumSkaters,  'J6ID': [None]*NumSkaters,
      'J7ID': [None]*NumSkaters,  'J8ID': [None]*NumSkaters,  'J9ID': [None]*NumSkaters, 
      'REFID': [None]*NumSkaters, 'TCID': [None]*NumSkaters, 'TSID': [None]*NumSkaters, 
      'ATSID': [None]*NumSkaters, 'skID': [None]*NumSkaters, 'eventID': [0]*NumSkaters}
## UPDATE eventID

dfProtocols = pd.DataFrame(data=dP) # Make Dataframe from dictionary

# Assign pID to dfProtocols. Depends on skID and eventID
dfProtocols['pID'] = dfProtocols.groupby(['Event_StartNum',
                                          'Event_Rank',
                                          'Event_Total_Score',
                                          'Component_Score',
                                          'eventID']).ngroup()

dfProtocols
### ***UPDATE: ATTACH judge ID's, TCID, etc. to events table or event_officials table.
### NOT here *** ###

,pID,Event_Rank,Event_StartNum,Event_Total_Score,Element_Score,Total_Element_Base_Value,Total_GOE_Points,Component_Score,Deductions,J1ID,...,J6ID,J7ID,J8ID,J9ID,REFID,TCID,TSID,ATSID,skID,eventID
0,8,1,17,81.21,44.14,31.80,12.34,37.07,0.0,None,...,None,None,None,None,None,None,None,None,None,0
1,2,2,11,79.24,42.73,31.70,11.03,36.51,0.0,None,...,None,None,None,None,None,None,None,None,None,0
2,3,3,12,75.51,41.28,31.00,10.28,34.23,0.0,None,...,None,None,None,None,None,None,None,None,None,0
3,10,4,19,73.96,39.70,30.60,9.10,34.26,0.0,None,...,None,None,None,None,None,None,None,None,None,0
4,4,5,13,73.08,38.87,31.40,7.47,34.21,0.0,None,...,None,None,None,None,None,None,None,None,None,0
5,6,6,15,69.99,36.95,28.70,8.25,33.04,0.0,None,...,None,None,None,None,None,None,None,None,None,0
6,9,7,18,68.67,34.61,29.40,5.21,35.06,1.0,None,...,None,None,None,None,None,None,None,None,None,0
7,7,8,16,67.29,34.40,27.70,6.70,32.89,0.0,None,...,None,None,None,None,None,None,None,None,None,0
8,5,9,14,66.93,35.81,32.70,3.11,32.12,1.0,None,...,None,None,None,None,None,None,None,None,None,0
9,15,10,6,64.28,34.77,30.60,4.17,29.51,0.0,None,...,None,None,None,None,None,None,None,None,None,0


#### Elements Table (performed elements)
This is a data frame of all performed elements.  
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique elemIDs.

In [23]:
# Start with just 1 skater, then expand to every skater in the event
#for j in range(NumSkaters) for i in range(NumElems)
dEL = {'elemID': [None]*NumElems*NumSkaters, 
       'Element_Number': [i+1 for j in range(NumSkaters) for i in range(NumElems)], 
       'Elem_Code': [element_details(df_body, j+1, i+1)[0] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'Info': [df_body[j].iloc[i+1][1] 
                 for j in range(NumSkaters) for i in range(NumElems)],
       'Base_Value': [df_body[j].iloc[i+1][2] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'GOE_Points': [df_body[j].iloc[i+1][3] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J1_score': [df_body[j].iloc[i+1][5] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J2_score': [df_body[j].iloc[i+1][6] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J3_score': [df_body[j].iloc[i+1][7] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J4_score': [df_body[j].iloc[i+1][8] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J5_score': [df_body[j].iloc[i+1][9] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J6_score': [df_body[j].iloc[i+1][10] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J7_score': [df_body[j].iloc[i+1][11] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J8_score': [df_body[j].iloc[i+1][12] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'J9_score': [df_body[j].iloc[i+1][13] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'Element_Points': [df_body[j].iloc[i+1][15] 
                     for j in range(NumSkaters) for i in range(NumElems)],
       'pID': [find_pID(j+1, eventID) 
                     for j in range(NumSkaters) for i in range(NumElems)] 
      }

dfElements = pd.DataFrame(data=dEL) # Make Dataframe from dictionary

# Assign pID to dfProtocols. Depends on skID and eventID
dfElements['elemID'] = dfElements.groupby(['Element_Number',
                                           'Elem_Code',
                                           'pID']).ngroup()

pd.set_option('display.max_rows', None) # Set to display all rows
display(dfElements)

,elemID,Element_Number,Elem_Code,Info,Base_Value,GOE_Points,J1_score,J2_score,J3_score,J4_score,J5_score,J6_score,J7_score,J8_score,J9_score,Element_Points,pID
0,14,1,3Tw4,NaN,6.00,2.66,4,4,4,5,5,4,4,5,5,8.66,8
1,28,2,3T,NaN,4.20,1.20,2,3,1,4,3,4,3,3,2,5.40,8
2,43,3,3LoTh,NaN,5.00,2.50,5,5,4,5,5,5,5,5,5,7.50,8
3,72,4,BoDs4,NaN,4.70,1.68,4,4,3,5,4,3,4,3,3,6.38,8
4,91,5,CCoSp4,NaN,3.50,1.35,3,4,4,4,5,4,3,4,4,4.85,8
5,97,6,4Li4,NaN,5.10,1.68,3,3,3,4,4,4,3,3,3,6.78,8
6,129,7,StSq3,NaN,3.30,1.27,3,4,2,4,5,4,4,4,4,4.57,8
7,18,1,BoDs3,NaN,4.30,1.41,3,2,4,3,5,3,4,3,3,5.71,2
8,26,2,3T,NaN,4.20,0.78,2,1,2,2,2,3,1,2,2,4.98,2
9,52,3,3Tw4,NaN,6.00,2.14,4,2,5,3,4,4,3,3,4,8.14,2


#### Element_Details Table
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique elem_detIDs.

In [24]:
# Create a list for each column of the table, only adding a row if the elem_code is not
# already in the table (in the elem_code list which will be added to the table)
dElD_Elem_Code = []
dElD_Name = []
dElD_Type = []
dElD_lvl = []
for j in range(NumSkaters):
    for i in range(NumElems):
        if(element_details(df_body, j+1, i+1)[0] not in dElD_Elem_Code):
            dElD_Elem_Code.append(element_details(df_body, j+1, i+1)[0])
            dElD_Name.append(element_details(df_body, j+1, i+1)[1])
            dElD_Type.append(element_details(df_body, j+1, i+1)[2])
            dElD_lvl.append(element_details(df_body, j+1, i+1)[3])

# Create dictionary from lists
dElD = {'Elem_Code': dElD_Elem_Code,
        'Name': dElD_Name, 
        'Type': dElD_Type,  
        'lvl': dElD_lvl
}

dfElement_Details = pd.DataFrame(data=dElD)    # Create data frame from dictionary

# Sort DataFrame By 'Type' and 'Elem_Code' columns
dfElement_Details = dfElement_Details.sort_values(['Type', 'Elem_Code'], 
                                                  ascending=[True, True])
dfElement_Details
### *** ADD BASE VALUE HERE? Take values from ISU Scale of Values?*** Or no -- 
# where to account for * 10% bonus? ###
## Adjust name in function used here, to give (for example) 3S and 3S< 
## different names? Maybe "Triple Sal Chow (cheated)" for example.

,Elem_Code,Name,Type,lvl
31,BoDs1,Back Outside Death Spiral,Death Spiral,1
27,BoDs2,Back Outside Death Spiral,Death Spiral,2
7,BoDs3,Back Outside Death Spiral,Death Spiral,3
3,BoDs4,Back Outside Death Spiral,Death Spiral,4
17,BoDsB,Back Outside Death Spiral,Death Spiral,Base
26,2A,Double Axel,Jump,None
20,2S,Double Sal Chow,Jump,None
19,2T,Double Toe Loop,Jump,None
22,3Lo,Triple Loop,Jump,None
15,3S,Triple Sal Chow,Jump,None


#### Program_Components Table
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique pcIDs.

In [25]:
dPC = {'pcID': [None]*NumPC*NumSkaters,
       # This (Component) and score locations will vary depending on the number of elements. Check that
       # this works for other events. Maybe make separate function if needed?
       'Component': [df_body[j].iloc[i+NumElems+3][0] 
                     for j in range(NumSkaters) for i in range(NumPC)], 
       'Factor': [df_body[j].iloc[i+NumElems+3][3] 
                  for j in range(NumSkaters) for i in range(NumPC)], 
       'J1_score': [df_body[j].iloc[i+NumElems+3][5] 
                    for j in range(NumSkaters) for i in range(NumPC)],
       'J2_score': [df_body[j].iloc[i+NumElems+3][6] 
                    for j in range(NumSkaters) for i in range(NumPC)], 
       'J3_score': [df_body[j].iloc[i+NumElems+3][7] 
                    for j in range(NumSkaters) for i in range(NumPC)], 
       'J4_score': [df_body[j].iloc[i+NumElems+3][8] 
                    for j in range(NumSkaters) for i in range(NumPC)], 
       'J5_score': [df_body[j].iloc[i+NumElems+3][9] 
                    for j in range(NumSkaters) for i in range(NumPC)], 
       'J6_score': [df_body[j].iloc[i+NumElems+3][10] 
                    for j in range(NumSkaters) for i in range(NumPC)],
       'J7_score': [df_body[j].iloc[i+NumElems+3][11] 
                    for j in range(NumSkaters) for i in range(NumPC)], 
       'J8_score': [df_body[j].iloc[i+NumElems+3][12] 
                    for j in range(NumSkaters) for i in range(NumPC)], 
       'J9_score': [df_body[j].iloc[i+NumElems+3][13] 
                    for j in range(NumSkaters) for i in range(NumPC)], 
       'Component_Score': [df_body[j].iloc[i+NumElems+3][15] 
                           for j in range(NumSkaters) for i in range(NumPC)], 
       'pID': [find_pID(j+1, eventID) 
                     for j in range(NumSkaters) for i in range(NumPC)]}

dfProgram_Components = pd.DataFrame(data=dPC) # Make Dataframe from dictionary

# Assign pcID to dfProgramComponents. Depends on pID and Component
dfProgram_Components['pcID'] = dfProgram_Components.groupby(['pID', 
                                                             'Component']).ngroup()

dfProgram_Components

,pcID,Component,Factor,J1_score,J2_score,J3_score,J4_score,J5_score,J6_score,J7_score,J8_score,J9_score,Component_Score,pID
0,43,Skating Skills,0.80,9.25,9.25,9.25,9.25,9.75,9.25,9.5,9.25,9,9.29,8
1,44,Transitions,0.80,9,9,8.75,9.5,9.5,9,8.75,9.25,9,9.07,8
2,42,Performance,0.80,9.25,9.5,9,9.5,9.5,9.25,9,9.5,9.25,9.32,8
3,40,Composition,0.80,9.25,9.25,9.25,9.5,9.5,9.5,8.75,9.25,9,9.29,8
4,41,Interpretation of the Music,0.80,9,9.5,9,9.5,9.5,9.5,9.25,9.5,9.25,9.36,8
5,13,Skating Skills,0.80,9.5,8.75,9.5,8.25,9.5,9,8.75,9,9.25,9.11,2
6,14,Transitions,0.80,9.25,8.5,9.25,8.5,9.5,8.75,9.25,9.25,9,9.04,2
7,12,Performance,0.80,9.5,8.5,9.25,8.5,9.75,9,8.75,9.5,9.5,9.14,2
8,10,Composition,0.80,9.5,8.5,9.5,8.5,9.75,9,9,9.25,9.25,9.14,2
9,11,Interpretation of the Music,0.80,9.25,8.75,9.5,8.5,9.5,9.25,9.25,9.25,9.25,9.21,2


#### Officials Table
Read in Officials Table from protocol url.  
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique oIDs.

In [26]:
dfO = pdf2table("Pairs Short", "official", page_number=11)

In [27]:
### WORK ON THIS
# Read in all pages for singles and pairs judges. 2 columns per page.
# Combine those into 1 table, in order
# Then from that table with the whole document, convert to clean data

tabula.read_pdf(officials_pdf_path, pages=1, stream=True,
                       area=[380, 66, 755, 189])

[                           ANDORRA
 0                              NaN
 1              International Judge
 2         Monica Lopez Camara, Ms.
 3                              NaN
 4                              NaN
 5                        AUSTRALIA
 6                              NaN
 7                      ISU Referee
 8              Lynch Susan M., Ms.
 9                              NaN
 10                       ISU Judge
 11        Alexandre Elizabeth, Ms.
 12             Andrew Rebecca, Ms.
 13      Clyde-Smith Angelique, Ms.
 14  du Preez Katherine Evelyn, Ms.
 15               Jelinek Lisa, Ms.
 16              Miller Joanna, Ms.
 17              Noyes Deborah, Ms.
 18            Rankin Peter K., Mr.
 19          Ryan Elizabeth W., Ms.
 20        Taylor Catherine L., Ms.
 21       Yacopetti Kylie Anne, Ms.
 22                             NaN
 23           International Referee
 24             Andrew Rebecca, Ms.
 25              Miller Joanna, Ms.
 26              Noyes Debor

In [28]:
# List of judges?
# https://www.isu.org/figure-skating/rules/fsk-communications/24797-2337-list-officials-fs-id-sys-2020-21/file
# https://www.isu.org/figure-skating/rules/fsk-communications/21860-isu-communication-2273/file

# Create list of names
Name_O = [dfO[0].iloc[i][1] for i in range(len(dfO[0].index))]
Name_tech = [dfO[1].iloc[i][1] for i in range(len(dfO[1].index))]
Name_O.extend(Name_tech)

# Create list of position types: Referee, then Judges, then TC, TS, ATS
# To use for position in data frame
PositionType_O = ["Referee"]
PositionType_O.extend(["Judge"]*(len(dfO[0].index)-1))
PositionType_tech = [dfO[1].iloc[i][0] for i in range(len(dfO[1].index))]
PositionType_O.extend(PositionType_tech)

# How to quantify officials' experience? Start date when they started judging?
dOFFICIAL = {'oID': [None]*NumOfficials, 
             'Name': Name_O, 
             'Position_Type': PositionType_O, 
             'Nation': [None]*NumOfficials, # Get Nation from separate isu doc***
             'Experience_Start': [None]*NumOfficials,
             'Status': [None]*NumOfficials,
             'Date_Updated': [None]*NumOfficials}

dfOfficials = pd.DataFrame(data=dOFFICIAL) # Make Dataframe from dictionary

# Assign oID to dfOfficials (based on Name, Position, and Nation)



dfOfficials

,oID,Name,Position_Type,Nation,Experience_Start,Status,Date_Updated
0,None,Ms. Beth CRANE,Referee,None,None,None,None
1,None,Ms. Elisabeth BINDER,Judge,None,None,None,None
2,None,Ms. Anja RIST,Judge,None,None,None,None
3,None,Ms. Wei SHI,Judge,None,None,None,None
4,None,Ms. Alice WALDER,Judge,None,None,None,None
5,None,Ms. Eva SZILAGYI,Judge,None,None,None,None
6,None,Ms. So Young AN,Judge,None,None,None,None
7,None,Ms. Joanna MILLER,Judge,None,None,None,None
8,None,Mr. Andre-Marc ALLAIN,Judge,None,None,None,None
9,None,Ms. Tomiko YAMADA,Judge,None,None,None,None


#### Event_Officials Table
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing.

In [29]:
# WIP: pdf 2 table for officials' names


Position_O = [dfO[0].iloc[i][0] for i in range(len(dfO[0].index))]
Position_tech = [dfO[1].iloc[i][0] for i in range(len(dfO[1].index))]
Position_O.extend(Position_tech)

dEvent_Officials = {'eventID': [0]*NumOfficials, 
                    'oID': [None]*NumOfficials, # *** Will need to find & input oID where name = name in spot on protocol
                    'Position': Position_O }#J1, J2, etc.

dfEvent_Officials = pd.DataFrame(data=dEvent_Officials) # Make Dataframe from dictionary
dfEvent_Officials

,eventID,oID,Position
0,0,None,Referee
1,0,None,Judge No.1
2,0,None,Judge No.2
3,0,None,Judge No.3
4,0,None,Judge No.4
5,0,None,Judge No.5
6,0,None,Judge No.6
7,0,None,Judge No.7
8,0,None,Judge No.8
9,0,None,Judge No.9


#### Coaches Table
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique coachIDs.

In [30]:
dCOACH = {'coachID': [], 
          'Name': [], 
          'DOB': [], }

dfCoaches = pd.DataFrame(data=dCOACH) # Make Dataframe from dictionary

#### Skater_Coaches Table (trID)
Define the dictionary, then make the dataframe from the dictionary.  
Fix anything that needs fixing. Then assign unique trIDs.

In [31]:
dSkater_Coach = {'trID': [], 
                 'skID': [], 
                 'coachID': [], 
                 'StartDate': [], 
                 'EndDate': [],
                 'TrainingLocation': []}

dfSkater_Coaches = pd.DataFrame(data=dSkater_Coach) # Make Dataframe from dictionary

In [32]:
# Plan: for each event, make a temp df (at least for dfSkaters, etc.) in this section. Then, append it to 
# a larger overall Skaters dataframe with everything from all events???

In [33]:
# List of all tables, in order:
tables = [dfSkaters, dfCompetitions, dfEvents, dfProtocols, dfElements, dfElement_Details,
          dfProgram_Components, dfOfficials, dfEvent_Officials, dfCoaches, dfSkater_Coaches]

In [34]:
tables

[    skID                                      Name Nation  Race   DOB
 0      5       Evgenia TARASOVA / Vladimir MOROZOV    RUS  None  None
 1     17                    Wenjing SUI / Cong HAN    CHN  None  None
 2      3                     Cheng PENG / Yang JIN    CHN  None  None
 3     12       Natalia ZABIIAKO / Alexander ENBERT    RUS  None  None
 4      7   Kirsten MOORE-TOWERS / Michael MARINARO    CAN  None  None
 5      0   Aleksandra BOIKOVA / Dmitrii KOZLOVSKII    RUS  None  None
 6     16             Vanessa JAMES / Morgan CIPRES    FRA  None  None
 7     13      Nicole DELLA MONICA / Matteo GUARISE    ITA  None  None
 8      2               Ashley CAIN / Timothy LEDUC    USA  None  None
 9     10     Minerva Fabienne HASE / Nolan SEEGERT    GER  None  None
 10    11           Miriam ZIEGLER / Severin KIEFER    AUT  None  None
 11     4             Evelyn WALSH / Trennt MICHAUD    CAN  None  None
 12    15                  Tae Ok RYOM / Ju Sik KIM    PRK  None  None
 13   

### Next Steps
Move Next Steps to Github Reademe instead. 

Make whole process from above inside of 1 wrapper--> give it event, pagenum, numSkaters, and it does everything.  
Extend to include all protocols etc. from Worlds 2019

### Work with pdfplumber instead of tabula to extract tables???